**IBM QISKIT CODE**

In [1]:
!pip install qiskit qiskit-aer pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.4 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=d0b3ca2a7dabcc0993e50223fcaa8abdb8a695999c48a58219cbfe9d0999446d
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc


In [3]:
pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.4 MB/s eta 0:00:00


**Deutsch-Jozsa Algorithm**

The Deutsch-Jozsa algorithm is a quantum algorithm designed to determine whether a given function $f:\{0,1\}^n \to \{0,1\}$ is **constant** (outputs the same value for all inputs) or **balanced** (outputs 0 for half the inputs and 1 for the other half) with a single evaluation, exponentially faster than any classical algorithm.

**Key steps:**

1. **Initialization:** Prepare $n$ qubits in the $|0\rangle$ state and one ancilla qubit in $|1\rangle$.
2. **Superposition:** Apply Hadamard gates to all qubits to create a superposition of all inputs.
3. **Oracle:** Implement the function $f$ as a quantum oracle that flips the phase depending on the function value.
4. **Interference:** Apply Hadamard gates again to the first $n$ qubits.
5. **Measurement:** Measure the first $n$ qubits — if all are zero, the function is constant; otherwise, it is balanced.

In [7]:
#Build Oracle from 4-bit function string
def custom_oracle(truth_table):
    n = 2
    oracle = QuantumCircuit(n + 1)
    for i in range(4):  # 2^2 = 4
        if truth_table[i] == '1':
            bin_input = format(i, '02b')
            for idx, bit in enumerate(bin_input):
                if bit == '0':
                    oracle.x(idx)
            oracle.mcx([0, 1], 2)  # Control on first 2, target = 3rd qubit
            for idx, bit in enumerate(bin_input):
                if bit == '0':
                    oracle.x(idx)
    return oracle

# ---- Deutsch-Jozsa Algorithm ----
def deutsch_jozsa_algorithm(oracle, n):
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.h(range(n + 1))
    qc.append(oracle, range(n + 1))
    qc.h(range(n))
    qc.measure(range(n), range(n))
    return qc


input_box = widgets.Text(
    value='0000',
    placeholder='Enter 4-bit string (e.g., 0000, 1010)',
    description='Truth Table:',
    style={'description_width': 'initial'}
)
run_button = widgets.Button(description='Run Deutsch-Jozsa')
output_area = widgets.Output()

def on_click(b):
    with output_area:
        clear_output()
        truth_table = input_box.value.strip()

        if len(truth_table) != 4 or any(c not in '01' for c in truth_table):
            print("❌ Please enter a valid 4-bit binary string (like 0000, 1010).")
            return

        zeros = truth_table.count('0')
        ones = truth_table.count('1')

        if zeros == 4:
            label = "constant-0"
        elif ones == 4:
            label = "constant-1"
        elif zeros == 2 and ones == 2:
            label = "balanced"
        else:
            label = "not valid for DJ"

        print(f"Input Truth Table: {truth_table} → Classified as: {label.upper()}")

        if label == "not valid for DJ":
            print("⚠️ The function is not valid for Deutsch-Jozsa Algorithm.")
            return

        oracle = custom_oracle(truth_table)
        dj_circuit = deutsch_jozsa_algorithm(oracle, 2)
        display(dj_circuit.draw('mpl'))


        simulator = AerSimulator()
        compiled = transpile(dj_circuit, simulator)
        result = simulator.run(compiled).result()
        counts = result.get_counts()
        print("Measurement Counts:", counts)

        if '00' in counts:
            print("✅ DJ Algorithm Output: CONSTANT function.")
        else:
            print("✅ DJ Algorithm Output: BALANCED function.")

run_button.on_click(on_click)
display(input_box, run_button, output_area)


Text(value='0000', description='Truth Table:', placeholder='Enter 4-bit string (e.g., 0000, 1010)', style=Desc…

Button(description='Run Deutsch-Jozsa', style=ButtonStyle())

Output()

| Label      | Type     | Description                               |
| ---------- | -------- | ----------------------------------------- |
| `oracle-A` | Constant | All outputs are `0`                       |
| `oracle-B` | Constant | All outputs are `1`                       |
| `oracle-C` | Balanced | `f(x) = x₀ ⊕ x₁ ⊕ x₂`                     |
| `oracle-D` | Balanced | `f(x) = x₀`                               |
| `oracle-E` | Balanced | `f(x) = x₁`                               |
| `oracle-F` | Balanced | `f(x) = x₀ AND x₁` (approximated)         |
| `oracle-G` | Balanced | Custom balanced using multiple CNOT gates |


In [6]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_bloch_multivector
from qiskit.quantum_info import partial_trace, Statevector
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt

def deutsch_jozsa_circuit(oracle: QuantumCircuit, n: int):
    qc = QuantumCircuit(n+1, n)
    qc.x(n)
    qc.h(n)
    for i in range(n):
        qc.h(i)
    qc.append(oracle, range(n+1))
    for i in range(n):
        qc.h(i)
    qc.measure(range(n), range(n))
    return qc

# Oracle Definitions
def oracle_A(n):  # constant-0
    return QuantumCircuit(n+1)

def oracle_B(n):  # constant-1
    qc = QuantumCircuit(n+1)
    qc.x(n)
    return qc

def oracle_C(n):  # balanced: f(x) = x0 XOR x1 XOR x2
    qc = QuantumCircuit(n+1)
    for i in range(n):
        qc.cx(i, n)
    return qc

def oracle_D(n):  # balanced: f(x) = x0
    qc = QuantumCircuit(n+1)
    qc.cx(0, n)
    return qc

def oracle_E(n):  # balanced: f(x) = x1
    qc = QuantumCircuit(n+1)
    qc.cx(1, n)
    return qc

def oracle_F(n):  # balanced: f(x) ≈ x0 AND x1 (Toffoli style)
    qc = QuantumCircuit(n+1)
    if n >= 2:
        qc.ccx(0, 1, n)
    return qc

def oracle_G(n):  # balanced: custom pattern with X gates
    qc = QuantumCircuit(n+1)
    qc.cx(0, n)
    qc.cx(2, n)
    return qc


oracle_map = {
    'oracle-A (constant 0)': oracle_A,
    'oracle-B (constant 1)': oracle_B,
    'oracle-C (x0⊕x1⊕x2)': oracle_C,
    'oracle-D (x0 only)': oracle_D,
    'oracle-E (x1 only)': oracle_E,
    'oracle-F (x0 AND x1)': oracle_F,
    'oracle-G (x0 XOR x2)': oracle_G
}


dropdown = widgets.Dropdown(
    options=list(oracle_map.keys()),
    value='oracle-A (constant 0)',
    description='Function (Oracle):',
    style={'description_width': 'initial'}
)

button = widgets.Button(description="Run Deutsch-Jozsa")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        n = 3
        selected_label = dropdown.value
        oracle_function = oracle_map[selected_label]
        oracle = oracle_function(n)

        dj_circuit = deutsch_jozsa_circuit(oracle, n)
        display(dj_circuit.draw('mpl'))

        simulator = AerSimulator()
        compiled = transpile(dj_circuit, simulator)
        result = simulator.run(compiled).result()
        counts = result.get_counts()

        if '0'*n in counts:
            actual_result = "constant"
        else:
            actual_result = "balanced"

        print(f"\nDeutsch-Jozsa Algorithm Result: The function is {actual_result.upper()}.")
        print(f"Oracle selected: {selected_label}")

        print("\nBloch vectors of input qubits before measurement:")
        dj_nom = dj_circuit.remove_final_measurements(inplace=False)
        state = Statevector.from_instruction(dj_nom)
        for qubit in range(n):
            traced = partial_trace(state, [i for i in range(n+1) if i != qubit])
            display(plot_bloch_multivector(traced, title=f"Bloch Vector of Qubit {qubit}"))

button.on_click(on_button_clicked)
display(dropdown, button, output)


Dropdown(description='Function (Oracle):', options=('oracle-A (constant 0)', 'oracle-B (constant 1)', 'oracle-…

Button(description='Run Deutsch-Jozsa', style=ButtonStyle())

Output()

**PENNYLANE CODE**

In [8]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 104.8 MB/s eta 0:00:00


In [9]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

n = 3  # Number of input qubits
dev = qml.device("default.qubit", wires=n+1, shots=1024)

# Define oracles
def oracle_A():  # Constant 0
    pass

def oracle_B():  # Constant 1
    qml.PauliX(wires=n)

def oracle_C():  # x0⊕x1⊕x2
    for i in range(n):
        qml.CNOT(wires=[i, n])

def oracle_D():  # x0 only
    qml.CNOT(wires=[0, n])

def oracle_E():  # x1 only
    qml.CNOT(wires=[1, n])

def oracle_F():  # x0 AND x1
    qml.Toffoli(wires=[0, 1, n])

def oracle_G():  # x0 XOR x2
    qml.CNOT(wires=[0, n])
    qml.CNOT(wires=[2, n])

oracle_map = {
    'oracle-A (constant 0)': oracle_A,
    'oracle-B (constant 1)': oracle_B,
    'oracle-C (x0⊕x1⊕x2)': oracle_C,
    'oracle-D (x0 only)': oracle_D,
    'oracle-E (x1 only)': oracle_E,
    'oracle-F (x0 AND x1)': oracle_F,
    'oracle-G (x0 XOR x2)': oracle_G
}

def deutsch_jozsa(oracle_func):
    @qml.qnode(dev)
    def circuit():
        for i in range(n):
            qml.Hadamard(wires=i)
        qml.PauliX(wires=n)
        qml.Hadamard(wires=n)

        oracle_func()

        for i in range(n):
            qml.Hadamard(wires=i)
        return [qml.expval(qml.PauliZ(wires=i)) for i in range(n)]

    exp_vals = circuit()

    bitstring = ''.join('0' if val > 0 else '1' for val in exp_vals)

    if bitstring == '0' * n:
        return "CONSTANT"
    else:
        return "BALANCED"


dropdown = widgets.Dropdown(
    options=list(oracle_map.keys()),
    value='oracle-A (constant 0)',
    description='Function (Oracle):',
    style={'description_width': 'initial'}
)
button = widgets.Button(description="Run Deutsch-Jozsa")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        selected = dropdown.value
        oracle_func = oracle_map[selected]
        result = deutsch_jozsa(oracle_func)
        print(f"Deutsch-Jozsa Result: The function is {result}")
        print(f"Selected Oracle: {selected}")

button.on_click(on_button_clicked)
display(dropdown, button, output)


/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pennylane/devices/device_api.py:193: PennyLaneDeprecationWarning: Setting shots on device is deprecated. Please use the `set_shots` transform on the respective QNode instead.
  warnings.warn(


Dropdown(description='Function (Oracle):', options=('oracle-A (constant 0)', 'oracle-B (constant 1)', 'oracle-…

Button(description='Run Deutsch-Jozsa', style=ButtonStyle())

Output()

**GOOGLE CIRQ CODE**

In [10]:
!pip install cirq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 72.1 MB/s eta 0:00:00


In [11]:
import cirq
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

n = 3
input_qubits = [cirq.LineQubit(i) for i in range(n)]
target_qubit = cirq.LineQubit(n)
qubits = input_qubits + [target_qubit]

# Define oracles
def oracle_A():
    return []

def oracle_B():
    return [cirq.X(target_qubit)]

def oracle_C():
    return [cirq.CNOT(q, target_qubit) for q in input_qubits]

def oracle_D():
    return [cirq.CNOT(input_qubits[0], target_qubit)]

def oracle_E():
    return [cirq.CNOT(input_qubits[1], target_qubit)]

def oracle_F():
    return [cirq.TOFFOLI(input_qubits[0], input_qubits[1], target_qubit)]

def oracle_G():
    return [cirq.CNOT(input_qubits[0], target_qubit), cirq.CNOT(input_qubits[2], target_qubit)]

oracle_map = {
    'oracle-A (constant 0)': oracle_A,
    'oracle-B (constant 1)': oracle_B,
    'oracle-C (x0⊕x1⊕x2)': oracle_C,
    'oracle-D (x0 only)': oracle_D,
    'oracle-E (x1 only)': oracle_E,
    'oracle-F (x0 AND x1)': oracle_F,
    'oracle-G (x0 XOR x2)': oracle_G
}

def deutsch_jozsa(oracle_fn):
    circuit = cirq.Circuit()
    circuit.append([cirq.H(q) for q in input_qubits])
    circuit.append([cirq.X(target_qubit), cirq.H(target_qubit)])
    circuit.append(oracle_fn())
    circuit.append([cirq.H(q) for q in input_qubits])
    circuit.append(cirq.measure(*input_qubits, key='result'))

    simulator = cirq.Simulator()
    result = simulator.run(circuit, repetitions=1)
    measured = result.measurements['result'][0]

    if all(x == 0 for x in measured):
        return "CONSTANT", circuit
    else:
        return "BALANCED", circuit

# UI
dropdown = widgets.Dropdown(
    options=list(oracle_map.keys()),
    value='oracle-A (constant 0)',
    description='Function (Oracle):',
    style={'description_width': 'initial'}
)
button = widgets.Button(description="Run Deutsch-Jozsa")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        clear_output()
        selected = dropdown.value
        oracle_fn = oracle_map[selected]
        result, circuit = deutsch_jozsa(oracle_fn)
        print(f"Deutsch-Jozsa Result: The function is {result}")
        print(f"Selected Oracle: {selected}")
        print(circuit)

button.on_click(on_button_clicked)
display(dropdown, button, output)


Dropdown(description='Function (Oracle):', options=('oracle-A (constant 0)', 'oracle-B (constant 1)', 'oracle-…

Button(description='Run Deutsch-Jozsa', style=ButtonStyle())

Output()